In [46]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeled_train_clean_reviews_stemmed.csv'
TEST_DF = BASE_DIR + 'test_clean_reviews_stemmed.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sampleSubmission.csv', 'labeled_train_clean_reviews.csv', 'word2vec_model_300dim_40minwords_10context_stemmed', 'labeledTrainData.tsv', 'sentences_for_word2vec.csv', 'test_submission.csv', 'testData.tsv', 'unlabeledTrainData.tsv', 'test_clean_reviews.csv']


In [47]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0)
test = pd.read_csv(TEST_DF, header = 0)
print "Read %d labeled train reviews" \
          "and %d test reviews" % (labeled_train["review"].size, test["review"].size)

IOError: File ../input/labeled_train_clean_reviews_stemmed.csv does not exist

Data leakage

Check if test["sentiment"] is correct

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

In [5]:
from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context"

word2vec_model = Word2Vec.load(WORD2VEC_MODEL_FILE)

# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
word2vec_model.init_sims(replace=True)

print("Number of words in the word2vec model vocabulary: %d" % len(word2vec_model.wv.vocab))

Number of words in the word2vec model vocabulary: 16490


Prepare

In [6]:
train_clean_reviews = []
# Getting clean reviews from training set
counter = 0.
for review in labeled_train["review"]:
    clean_review = review_to_clean_review(review, remove_numbers=False)
    train_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

# train_clean_reviews = map(
#     lambda review: review_to_clean_review(review, remove_numbers=False),
#     labeled_train["review"])

test_clean_reviews = []
# Getting clean review from testing set
counter = 0.
for review in test["review"]:
    clean_review = review_to_clean_review(review, remove_numbers=False)
    test_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Test review %d of %d" % (counter, len(test["review"]))
    counter = counter + 1.

# test_clean_reviews = map(
#     lambda review: review_to_clean_review(review, remove_numbers=False),
#     test["review"])

all_clean_reviews = train_clean_reviews + test_clean_reviews

Train review 0 of 25000
Train review 5000 of 25000
Train review 10000 of 25000
Train review 15000 of 25000
Train review 20000 of 25000
Test review 0 of 25000
Test review 5000 of 25000
Test review 10000 of 25000
Test review 15000 of 25000
Test review 20000 of 25000


In [7]:
# print(train_clean_reviews[0:2])
# print(test_clean_reviews[0:2])

In [8]:
# We vectorize the text corpus by turning each text into a sequence of integers
# Each integer is the index of a token in the dictionary
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS_FOR_KERAS_TOKENIZER = 200000
#
# num_words: the maximum number of words to keep, based on frequency.
keras_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS_FOR_KERAS_TOKENIZER)
#
# fit_on_texts accepts a list of strings, a generator of strings or 
# a list of list of strings. In the last case, it assumes each entry of the lists to be a token.
# Here we provide a list of strings.
keras_tokenizer.fit_on_texts(all_clean_reviews)
word_index = keras_tokenizer.word_index
print('Keras Tokenizer found %s unique tokens' % len(word_index))
#
# texts_to_sequences transforms each text in texts to a sequence of integers.
train_sequences = keras_tokenizer.texts_to_sequences(train_clean_reviews)
test_sequences = keras_tokenizer.texts_to_sequences(test_clean_reviews)

Using TensorFlow backend.


Keras Tokenizer found 101376 unique tokens


In [41]:
print(np.random.choice(word_index.keys(), 1000))

[u'mezzanine' u'douche' u'nimbar' u'washroom' u'stoke' u'cryptic' u'ocar'
 u'kywildflower' u'revisionists' u'vasty' u'gaughan' u'unbowed' u'sonya'
 u'philharmagic' u'troublesome' u'telescoping' u'mooching' u'terrorizers'
 u'bubbling' u'naysayers' u'encyclopedic' u'criticizes' u'maraglia'
 u'arkin' u'pilloried' u'intimacy' u'bookcase' u'shadix' u'unsurvivable'
 u'skunked' u'topiary' u'catapulted' u'falsified' u'mrudul'
 u'contaminants' u'braggart' u'castrating' u'unveiled' u'watchowski'
 u'maude' u'freeform' u'subconsciously' u'brugge' u'monette' u'constained'
 u'mitsugoro' u'labouredly' u'homestretch' u'hierarchy' u'commandeering'
 u'jelling' u'materializer' u'lards' u'stupefy' u'mainframes' u'gazes'
 u'fok' u'olympians' u'stepan' u'sleestak' u'overdramatized' u'gawkers'
 u'bonding' u'arthurs' u'equalled' u'galatea' u'cookies' u'sobbing'
 u'cocco' u'anthropomorhic' u'purticularly' u'c' u'zappati' u'unlicensed'
 u'francophiles' u'jenniferbeals' u'reevaluate' u'bustiers' u'witful'
 u'ham

In [9]:
# We pad all text sequences to the same length.
# By default zeros are padded at the front.
from keras.preprocessing.sequence import pad_sequences

# Set max length for each review sequence.
MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN = 500

train_pad_sequences = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)
test_pad_sequences = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)

In [44]:
# Prepare word embedding matrix

# Choose the smaller number of the two as column length of the matrix
num_words = min(MAX_NUM_WORDS_FOR_KERAS_TOKENIZER, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
words_not_found = []
for word, i in word_index.items():
    if word in word2vec_model.wv.vocab:
        embedding_matrix[i] = word2vec_model.wv.get_vector(word)
    else:
        words_not_found.append(word)
# Null word embeddings are words that don't exist in the embedding matrix
# and are therefore represented as zero vectors.
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 84920


In [45]:
print(np.random.choice(words_not_found, 1000))

[u'cockily' u'ricochet' u'antebellum' u'shepherdesses' u'jaundiced'
 u'meiyan' u'mackie' u'crackd' u'newmar' u'aviva' u'alternated'
 u'straightedge' u'delaware' u'krog' u'cocteau' u'gerardo' u'catillon'
 u'shortest' u'cariie' u'hoovered' u'bannacheck' u'zabalza' u'reined'
 u'glendale' u'tamarind' u'tipple' u'miamis' u'colluded' u'initself'
 u'paunchy' u'vel' u'disneyish' u'gerards' u'tarn' u'rosana' u'vanne'
 u'hibernia' u'hollinghurst' u'understanda' u'lightsabre' u'fantasma'
 u'shriveled' u'crispon' u'xtianity' u'ruggles' u'gushed' u'actally'
 u'martijn' u'zoeys' u'allcharacters' u'mua' u'attilla' u'mabuse'
 u'petitions' u'gazelles' u'antonik' u'concussive' u'opinionated'
 u'salinger' u'swink' u'xp' u'accumulator' u'mammothly' u'bosomy' u'prs'
 u'keeanu' u'chakotay' u'comraderie' u'omaha' u'linguine' u'ombra'
 u'inferiors' u'dwervick' u'disconsolate' u'valientes' u'sahib' u'ganges'
 u'experimentality' u'sens' u'rambunctious' u'froqu' u'alva'
 u'steenburgen' u'weaselly' u'overhauling'

In [11]:
# Split train_sequences into train and validation. Ratio: 80/20
VALIDATION_SPLIT = 0.2
np.random.seed(1234)

# 
perm = np.random.permutation(len(train_sequences))
index_train = perm[:int(len(train_sequences)*(1-VALIDATION_SPLIT))]
index_val = perm[int(len(train_sequences)*(1-VALIDATION_SPLIT)):]

x_train = train_pad_sequences[index_train]
x_val = train_pad_sequences[index_val]
y_train = labeled_train["sentiment"][index_train].tolist()
y_val = labeled_train["sentiment"][index_val].tolist()

print('Randomly split %d pad sequences for training, %d for validation' % (len(x_train) ,len(x_val)))


Randomly split 20000 pad sequences for training, 5000 for validation


In [12]:
x_test = test_pad_sequences
y_test = test["sentiment"]

## Model Architecture

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Conv1D, MaxPooling1D ,GlobalMaxPooling1D

In [14]:
def layer1_cnn(dropout=0.2, num_filters=64, kernel_size=2):
    model = Sequential()

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [15]:
def layer3_cnn():
    model = Sequential()
    
    NUM_FILTERS = 64

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [16]:
layer1_cnn_model = layer1_cnn()
layer1_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          30413100  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 499, 64)           38464     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 30,453,677
Trainable params: 40,577
Non-trainable params: 30,413,100
___________________________________________________________

In [17]:
layer1_cnn_model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 255us/step - loss: 0.4418 - acc: 0.7960 - val_loss: 0.3181 - val_acc: 0.8682
Epoch 2/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.2932 - acc: 0.8796 - val_loss: 0.3031 - val_acc: 0.8720
Epoch 3/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2594 - acc: 0.8953 - val_loss: 0.2861 - val_acc: 0.8792
Epoch 4/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2230 - acc: 0.9132 - val_loss: 0.2588 - val_acc: 0.8944
Epoch 5/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.1964 - acc: 0.9245 - val_loss: 0.2571 - val_acc: 0.8944
Epoch 6/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.1746 - acc: 0.9335 - val_loss: 0.2606 - val_acc: 0.8948
Epoch 7/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.1526 - acc: 0.9444 - val_loss: 0.2644 - v

In [18]:
layer3_cnn_model = layer3_cnn()
layer3_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 300)          30413100  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 499, 64)           38464     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 249, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 248, 64)           8256      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 123, 64)           8256      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
__________

In [19]:
layer3_cnn_model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 6s 307us/step - loss: 0.4339 - acc: 0.7896 - val_loss: 0.3334 - val_acc: 0.8586
Epoch 2/10
20000/20000 [==============================] - 5s 247us/step - loss: 0.2989 - acc: 0.8774 - val_loss: 0.3070 - val_acc: 0.8736
Epoch 3/10
20000/20000 [==============================] - 5s 248us/step - loss: 0.2567 - acc: 0.8985 - val_loss: 0.2905 - val_acc: 0.8804
Epoch 4/10
20000/20000 [==============================] - 5s 248us/step - loss: 0.2339 - acc: 0.9083 - val_loss: 0.2764 - val_acc: 0.8862
Epoch 5/10
20000/20000 [==============================] - 5s 249us/step - loss: 0.1972 - acc: 0.9257 - val_loss: 0.2814 - val_acc: 0.8846
Epoch 6/10
20000/20000 [==============================] - 5s 249us/step - loss: 0.1693 - acc: 0.9397 - val_loss: 0.2913 - val_acc: 0.8874
Epoch 7/10
20000/20000 [==============================] - 5s 249us/step - loss: 0.1464 - acc: 0.9490 - val_loss: 0.3399 - v

In [20]:
layer1_cnn_model_dropout5 = layer1_cnn(dropout=0.5)
layer1_cnn_model_dropout5.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 4s 212us/step - loss: 0.5096 - acc: 0.7496 - val_loss: 0.3463 - val_acc: 0.8566
Epoch 2/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.3413 - acc: 0.8613 - val_loss: 0.2962 - val_acc: 0.8760
Epoch 3/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.2897 - acc: 0.8867 - val_loss: 0.2728 - val_acc: 0.8880
Epoch 4/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2601 - acc: 0.9014 - val_loss: 0.2664 - val_acc: 0.8876
Epoch 5/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2373 - acc: 0.9133 - val_loss: 0.2612 - val_acc: 0.8920
Epoch 6/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2167 - acc: 0.9229 - val_loss: 0.2621 - val_acc: 0.8944
Epoch 7/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.1992 - acc: 0.9290 - val_loss: 0.2579 - v

In [21]:
y_test_pred_layer1_cnn = layer1_cnn_model.predict(x_test)
y_test_pred_layer1_cnn_dropout5 = layer1_cnn_model_dropout5.predict(x_test)
y_test_pred_layer3_cnn = layer3_cnn_model.predict(x_test)

In [22]:
def to_binary(predicts):
    return map(lambda predict: 1 if predict > 0.5 else 0, predicts)

In [23]:
y_test_pred_layer1_cnn = to_binary(y_test_pred_layer1_cnn)
y_test_pred_layer1_cnn_dropout5 = to_binary(y_test_pred_layer1_cnn_dropout5)
y_test_pred_layer3_cnn = to_binary(y_test_pred_layer3_cnn)

In [24]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

In [25]:
def print_auc_score(model_name, y_test_pred):
    print("The AUC score for %s is : %.4f." % (model_name, roc_auc_score(y_test, y_test_pred)))
    return

In [26]:
print_auc_score("layer1 cnn", y_test_pred_layer1_cnn)
print_auc_score("layer1 cnn dropout 0.5", y_test_pred_layer1_cnn_dropout5)
print_auc_score("layer3 cnn", y_test_pred_layer3_cnn)

The AUC score for layer1 cnn is : 0.8946.
The AUC score for layer1 cnn dropout 0.5 is : 0.8958.
The AUC score for layer3 cnn is : 0.8770.


In [27]:
layer1_cnn_32f_5d = layer1_cnn(dropout=0.5, num_filters=128)
layer1_cnn_32f_5d.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 272us/step - loss: 0.4690 - acc: 0.7767 - val_loss: 0.3150 - val_acc: 0.8688
Epoch 2/10
20000/20000 [==============================] - 5s 247us/step - loss: 0.3098 - acc: 0.8764 - val_loss: 0.2832 - val_acc: 0.8804
Epoch 3/10
20000/20000 [==============================] - 5s 246us/step - loss: 0.2658 - acc: 0.8969 - val_loss: 0.2642 - val_acc: 0.8904
Epoch 4/10
20000/20000 [==============================] - 5s 246us/step - loss: 0.2322 - acc: 0.9105 - val_loss: 0.2500 - val_acc: 0.8952
Epoch 5/10
20000/20000 [==============================] - 5s 247us/step - loss: 0.2009 - acc: 0.9278 - val_loss: 0.2478 - val_acc: 0.8940
Epoch 6/10
20000/20000 [==============================] - 5s 247us/step - loss: 0.1788 - acc: 0.9364 - val_loss: 0.2539 - val_acc: 0.8998
Epoch 7/10
20000/20000 [==============================] - 5s 246us/step - loss: 0.1469 - acc: 0.9519 - val_loss: 0.2592 - v

In [28]:
def layer1_cnn_d(dropout=0.2, num_filters=64, kernel_size=2):
    model = Sequential()

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
#     model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(MaxPooling1D(2))
#     model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [29]:
layer1_cnn_d_0 = layer1_cnn_d()
layer1_cnn_d_0.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 4s 219us/step - loss: 0.5107 - acc: 0.7386 - val_loss: 0.3429 - val_acc: 0.8556
Epoch 2/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.3586 - acc: 0.8453 - val_loss: 0.3032 - val_acc: 0.8696
Epoch 3/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.3223 - acc: 0.8645 - val_loss: 0.2859 - val_acc: 0.8776
Epoch 4/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2897 - acc: 0.8799 - val_loss: 0.2887 - val_acc: 0.8780
Epoch 5/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2607 - acc: 0.8947 - val_loss: 0.2709 - val_acc: 0.8850
Epoch 6/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2495 - acc: 0.9004 - val_loss: 0.2855 - val_acc: 0.8768
Epoch 7/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2259 - acc: 0.9123 - val_loss: 0.2655 - v

In [30]:
y_test_pred_layer1_cnn_d_0 = layer1_cnn_d_0.predict(x_test)
y_test_pred_layer1_cnn_d_0 = to_binary(y_test_pred_layer1_cnn_d_0)
print_auc_score("layer1 cnn_d", y_test_pred_layer1_cnn_d_0)

The AUC score for layer1 cnn_d is : 0.8910.


In [31]:
layer1_cnn_d_5d = layer1_cnn_d(dropout=0.5)
layer1_cnn_d_5d.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 4s 221us/step - loss: 0.6255 - acc: 0.6331 - val_loss: 0.4290 - val_acc: 0.8296
Epoch 2/20
20000/20000 [==============================] - 4s 193us/step - loss: 0.4607 - acc: 0.7943 - val_loss: 0.3544 - val_acc: 0.8548
Epoch 3/20
20000/20000 [==============================] - 4s 192us/step - loss: 0.3976 - acc: 0.8306 - val_loss: 0.3178 - val_acc: 0.8698
Epoch 4/20
20000/20000 [==============================] - 4s 192us/step - loss: 0.3594 - acc: 0.8496 - val_loss: 0.3022 - val_acc: 0.8750
Epoch 5/20
20000/20000 [==============================] - 4s 193us/step - loss: 0.3376 - acc: 0.8623 - val_loss: 0.3034 - val_acc: 0.8728
Epoch 6/20
20000/20000 [==============================] - 4s 192us/step - loss: 0.3188 - acc: 0.8688 - val_loss: 0.2804 - val_acc: 0.8810
Epoch 7/20
20000/20000 [==============================] - 4s 192us/step - loss: 0.3037 - acc: 0.8781 - val_loss: 0.2817 - v

In [32]:
y_test_pred_layer1_cnn_d_5d = layer1_cnn_d_5d.predict(x_test)
y_test_pred_layer1_cnn_d_5d = to_binary(y_test_pred_layer1_cnn_d_5d)
print_auc_score("layer1 cnn_d", y_test_pred_layer1_cnn_d_5d)

The AUC score for layer1 cnn_d is : 0.8926.


In [33]:
def layer3_cnn_1p(dropout=0.2):
    model = Sequential()
    
    NUM_FILTERS = 64

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [34]:
layer3_cnn_1p_0 = layer3_cnn_1p(dropout=0.5)
layer3_cnn_1p_0.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 7s 346us/step - loss: 0.4937 - acc: 0.7537 - val_loss: 0.3577 - val_acc: 0.8416
Epoch 2/20
20000/20000 [==============================] - 6s 301us/step - loss: 0.3437 - acc: 0.8643 - val_loss: 0.2932 - val_acc: 0.8764
Epoch 3/20
20000/20000 [==============================] - 6s 301us/step - loss: 0.2962 - acc: 0.8852 - val_loss: 0.2764 - val_acc: 0.8874
Epoch 4/20
20000/20000 [==============================] - 6s 302us/step - loss: 0.2589 - acc: 0.8980 - val_loss: 0.2732 - val_acc: 0.8852
Epoch 5/20
20000/20000 [==============================] - 6s 302us/step - loss: 0.2345 - acc: 0.9125 - val_loss: 0.2707 - val_acc: 0.8846
Epoch 6/20
20000/20000 [==============================] - 6s 300us/step - loss: 0.2161 - acc: 0.9182 - val_loss: 0.2760 - val_acc: 0.8894
Epoch 7/20
20000/20000 [==============================] - 6s 302us/step - loss: 0.1923 - acc: 0.9281 - val_loss: 0.2798 - v